# 2.0 CCLE Gene Expression Signature

In [29]:
from clustergrammer2 import net

In [30]:
import clustergrammer_groupby as cby
import warnings
warnings.filterwarnings('ignore')

In [175]:
import numpy as np
import pandas as pd
df = pd.read_csv('../data/CCLE/CCLE.txt.gz', compression='gzip', index_col=0)

In [ ]:
net.load_df(df)
net.filter_N_top(inst_rc='row', N_top=1000, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
df_tmp = net.export_df().round(2)
net.enrichrgram('KEA_2015')
net.widget()

In [34]:
df.shape

(18874, 1037)

In [32]:
from ast import literal_eval as make_tuple
cols = df.columns.tolist()
new_cols = [make_tuple(x) for x in cols]
df.columns = new_cols

In [33]:
all_genes = df.index.tolist()
len(all_genes)

18874

# CCLE Gene Expression Data, Z-score Genes

In [172]:
rows = df.index.tolist()
new_rows = [(x,) for x in rows]
df.index = new_rows
net.load_df(df)
net.filter_N_top(inst_rc='row', N_top=1000, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
df_tmp = net.export_df().round(2)
net.load_df(df_tmp)
net.cluster()
net.dendro_cats(axis='row', dendro_level=5)
df_dendro = net.export_df()
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "KRT19", "ini": 1000, "clust": 967, "rank": 948, "rankvar": 689…

In [150]:
rows = df_dendro.index.tolist()
rows[0]

('KRT19', 'Group 5: cat-13')

In [151]:
gene_list = sorted([x[0] for x in rows if x[1] == 'Group 5: cat-5'])
len(gene_list)

121

### Tissue Type Signatures

In [152]:
gmts = {}
gmts['kea'] = net.load_gmt('../data/Enrichr_Libraries_of_Interest/KEA_2015.txt')
gmts['go-process'] = net.load_gmt('../data/Enrichr_Libraries_of_Interest/GO_Biological_Process_2018.txt')

In [153]:
len(all_genes)

18874

In [154]:
len(set(all_genes).intersection(gene_list))

121

In [155]:
from scipy.stats import binom_test
def enrich_gene_list_using_lib(lib_json, gene_list, background_list, pval_cutoff=0.05):
    len_gene_list = len(gene_list)
    
    list_terms = []
    list_pval = []
    # list of series that will be used to make dataframe
    list_term_ser = []
    for inst_term in lib_json:
        
        term_ser = pd.Series(data=np.zeros(len(gene_list)), index=gene_list)
        
        term_genes = lib_json[inst_term]
        
        p_expect = len(set(all_genes).intersection(term_genes))/len(all_genes)
        
        found_genes = list(set(gene_list).intersection(term_genes))
        actual_k = len(found_genes)
        
        # set found genes to one
        term_ser[found_genes] =  1
        
        if actual_k/len_gene_list > p_expect:
            inst_pval = binom_test(actual_k, len_gene_list, p_expect)
        else:
            inst_pval = 0.5
        
        if inst_pval < pval_cutoff:

            term_name = (inst_term, 'Pval: ' + str(inst_pval))
            
            list_terms.append(term_name)
            list_pval.append(inst_pval)

            term_ser.name = term_name

            list_term_ser.append(term_ser)
            
    ser_pval = pd.Series(data=list_pval, index=list_terms).sort_values()
    df_term = pd.concat(list_term_ser, axis=1)
    
    # remove genes (rows) with no matches
    gene_sum = df_term.sum(axis=1)
    keep_genes = gene_sum[gene_sum > 0].index.tolist()
    df_term = df_term.loc[keep_genes]
    
    return ser_pval, df_term

In [167]:
genes_1k = [x[0] for x in rows]

# KEA

In [168]:
ser_pval, df_term = enrich_gene_list_using_lib(gmts['kea'], genes_1k, all_genes)

In [169]:
keep_top_enr = ser_pval.index.tolist()[:10]

In [171]:
net.load_df(df_term[keep_top_enr])
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "ABCA1", "ini": 127, "clust": 36, "rank": 109, "rankvar": 116, …

# GO Biological Process

In [160]:
ser_pval, df_term = enrich_gene_list_using_lib(gmts['go-process'], genes_1k, all_genes, 0.01)
ser_pval.shape
df_term.shape

(672, 483)

In [161]:
keep_top_enr = ser_pval.index.tolist()[:15]

In [162]:
net.load_df(df_term[keep_top_enr])
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "ADAMTS1", "ini": 335, "clust": 289, "rank": 228, "rankvar": 23…

In [163]:
len(gene_list)

121

In [7]:
df_sig, keep_genes_dict, df_gene_pval, fold_info = cby.generate_signatures(df,
                                                                     'tissue', num_top_dims=100)

net.load_df(df_sig)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "ABCA1", "ini": 703, "clust": 342, "rank": 341, "rankvar": 485,…

### Add tissue category to genes

In [8]:
gene_sig = df_sig.idxmax(axis=1)
gs_dict = {}
for inst_gene in gene_sig.index.tolist():
    gs_dict[inst_gene] = gene_sig[inst_gene][0]
    
rows = df.index.tolist()
new_rows = [(x, 'Cell Type: ' + gs_dict[x]) if x in gs_dict else (x, 'N.A.') for x in rows ]
df.index = new_rows
net.load_df(df)

In [9]:
ct_color = net.viz['cat_colors']['col']['cat-0']

In [10]:
def set_cat_colors(axis, cat_index, cat_title=False):
    for inst_ct in ct_color:
        if cat_title != False:
            cat_name = cat_title + ': ' + inst_ct
        else:
            cat_name = inst_ct
            
        inst_color = ct_color[inst_ct]
        net.set_cat_color(axis=axis, cat_index=cat_index, cat_name=cat_name, inst_color=inst_color)

In [11]:
set_cat_colors('row', 1)

### CCLE Data with Gene-Tissue Category

In [12]:
net.load_df(df)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "KRT19", "ini": 1000, "clust": 967, "rank": 948, "rankvar": 689…